## Latent Dirichlet Allocation

TODO, Explanation of how LDA works to generate vectors from sentences....

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df_variants = pd.read_csv('../data/training_variants.csv',index_col=0)

In [4]:
tokenized_data = pd.read_pickle('../data/df_pk.pk')
tokenized_data.head()

,text
0,"[cyclin-depend, kinas, cdk, regul, varieti, fu..."
1,"[abstract, background, non-smal, cell, lung, c..."
2,"[abstract, background, non-smal, cell, lung, c..."
3,"[recent, evid, demonstr, acquir, uniparent, di..."
4,"[oncogen, mutat, monomer, casita, b-lineag, ly..."


In [44]:
def return_same(ls):
    return ls
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import Pipeline
pipe = Pipeline([('encoder',CountVectorizer(analyzer=return_same,max_df=.5,
                                   min_df=50)),('lda',LatentDirichletAllocation(n_topics=3, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0))])


In [47]:
y = df_variants.Class[:2000]

In [45]:
X = tokenized_data.text.values
x_tf = pipe.fit_transform(X[:2000])
x_tf[:10]

array([[  3.10193498e-04,   5.20408012e-01,   4.79281794e-01],
       [  2.79676155e-01,   6.20330980e-01,   9.99928648e-02],
       [  2.79676155e-01,   6.20330980e-01,   9.99928648e-02],
       [  1.01162157e-02,   9.89538655e-01,   3.45129436e-04],
       [  3.44600001e-01,   1.32845961e-01,   5.22554038e-01],
       [  3.44600001e-01,   1.32845961e-01,   5.22554038e-01],
       [  3.44600001e-01,   1.32845961e-01,   5.22554038e-01],
       [  4.52360290e-03,   9.95347929e-01,   1.28468272e-04],
       [  1.74931861e-01,   4.78142397e-01,   3.46925742e-01],
       [  3.59307165e-04,   8.40363930e-01,   1.59276763e-01]])

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
-cross_val_score(SVC(probability=True),x_tf,y,scoring='neg_log_loss')

array([ 1.57916671,  1.74207862,  1.57760988])

In [49]:
-cross_val_score(LogisticRegression(),x_tf,y,scoring='neg_log_loss')

array([ 1.6006102 ,  1.72516505,  1.58972676])

In [51]:
from sklearn.ensemble import RandomForestClassifier
-cross_val_score(RandomForestClassifier(n_estimators=1000),x_tf,y,scoring='neg_log_loss')

array([ 4.51564208,  6.08713479,  3.97203198])

In [53]:
y.value_counts()/y.size

7    0.2975
4    0.2165
1    0.1970
2    0.1410
5    0.0505
6    0.0430
3    0.0310
9    0.0175
8    0.0060
Name: Class, dtype: float64